In [1]:
using FilterDDP
using Random

Precompiling packages...
   2477.9 ms  ? DomainSets
   2654.7 ms  ? Symbolics
Info Given FilterDDP was explicitly requested, output will be shown live 
ERROR: Method overwriting is not permitted during Module precompilation. Use `__precompile__(false)` to opt-out of precompilation.
   2669.5 ms  ? FilterDDP
[ Info: Precompiling FilterDDP [ac7c0669-fec3-40a7-96c9-a64efa85c273]
ERROR: Method overwriting is not permitted during Module precompilation. Use `__precompile__(false)` to opt-out of precompilation.
[ Info: Skipping precompilation since __precompile__(false). Importing FilterDDP [ac7c0669-fec3-40a7-96c9-a64efa85c273].
Precompiling packages...
   2609.1 ms  ? DomainSets
Info Given Symbolics was explicitly requested, output will be shown live 
ERROR: Method overwriting is not permitted during Module precompilation. Use `__precompile__(false)` to opt-out of precompilation.
   3017.9 ms  ? Symbolics
[ Info: Precompiling Symbolics [0c5d862f-8b57-4792-8d23-62f2024744c7]
ERROR: Method ov

In [8]:
# set OCP parameters

nx = 2  # number of states
nu = 1  # number of controls

h = 1e-4  # dt, time discretisation
N = 1000  # no. timesteps, i.e., horizon of T = 0.1s
L = 50 * 1e-3
C = 100 * 1e-6
Vg = 50
R = 50

# stores solver parameters
options = Options{Float64}(verbose=true, optimality_tolerance=1e-8);

### 1. Setting the objective function

For each discrete timestep, we need to set an objective function using the ```Objective``` struct. The objective function is a scalar-valued function which takes as input a state vector and control vector. 

In [7]:
# set objective functions

Vrefs = [0.5 * Vg * sin(100 * pi * t) for t = LinRange(0.0, 0.1, N)];
objectives_ = [(x, u) -> (x[2] - Vref)^2 for Vref in Vrefs];
objectives = [Objective(objective, nx, nu) for objective in objectives_];

### 2. Setting the dynamics mappings

For each discrete timestep *except the last step*, we need to set the discrete-time dynamics mappings using the ```Dynamics``` struct. The dynamics mapping is a vector-valued function which takes as input a state vector and control vector and outputs the state at the next discrete timestep. 

In [9]:
# set discrete-time dynamics

f = (x, u) -> begin
    x1dot = -x[2] / L + Vg / L * (2 * u[1] - 1)
    x2dot = x[1] / C + x[2] / (R * C)
    return x + h * [x1dot; x2dot]
end
dynamics_ = Dynamics(f, nx, nu);
dynamics = [dynamics_ for k = 1:N-1];

### 3. Setting the constraint functions

For each discrete timestep, we need to set the bounds inequality constraints on the controls and nonlinear equality constraints using the ```Bounds``` and ```Constraints``` structs, respectively. For control limits, we set a lower and upper limit (this can be ```-Inf``` and ```Inf```). Note, we do not apply equality constraints in this example.

In [10]:
# set stage-wise constraints

# inequality constraints (bounds on controls)
bound = Bound([0.0], [1.0])  # u \in [0, 1]
bounds = [bound for k in 1:N]

# nonlinear equality constraints (none)
constraint = Constraint((x, u) -> [], nx, nu);  # no equality constraints
constraints = [constraint for k=1:N];

### 4. Initialise and solve

Finally, we initialise the FilterDDP solver with the dynamics, constraints, objectives and bounds structs. We then call the ```solve!``` function with an initial state ```x1``` and initial control vector ```ū```.

In [6]:
solver = Solver(Float64, dynamics, objectives, constraints, bounds, options=options)

x1 = [0.0; 0.0]
ū = [[0.01] for k = 1:N]  # initialise u \in [0, 1]
solve!(solver, x1, ū)

   
 _____ _ _ _            ____  ____  ____  
|  ___(_) | |_ ___ _ __|  _ \|  _ \|  _ \ 
| |_  | | | __/ _ \ '__| | | | | | | |_) |
|  _| | | | ||  __/ |  | |_| | |_| |  __/ 
|_|   |_|_|\__\___|_|  |____/|____/|_|                 
    
  iter     objective        pr_inf       du_inf       cs_inf     lg(μ)   lg(reg)    alpha     ls   wall_time  solver_time
     0   1.27821377e+14   0.0000e+00   1.8040e+13   9.9000e-01    0.00      -     0.0000e+00   0     0.00     0.00
     1   1.23858166e+14   0.0000e+00   1.7758e+13   9.9013e-01    0.00      -     1.5625e-02   0    17663.67    4686.94
     2   6.96702181e+13   0.0000e+00   1.3318e+13   9.8565e-01    0.00      -     2.5000e-01   0    17720.88    4691.57
     3   5.33412608e+13   0.0000e+00   1.1653e+13   9.8618e-01    0.00      -     1.2500e-01   0    17735.48    4695.68
     4   4.08394028e+13   0.0000e+00   1.0197e+13   9.8683e-01    0.00      -     1.2500e-01   0    17748.75    4699.87
     5   3.12676677e+13   0.0000e+00   8.9222e

([[0.0, 0.0], [0.09999999999585496, 0.0], [0.19999999999098514, 0.09999999999585496], [0.2997999999851538, 0.3019999999867572], [0.3991959999780136, 0.6078399999716462], [0.4979803199690366, 1.0191927999490926], [0.5959419343573844, 1.537556975917111], [0.6928668203896537, 2.1642500497928374], [0.7885383202674928, 2.9004018711783477], [0.8827375164909252, 3.7469482288694076]  …  [0.8978646622946469, -6.981572082318167], [0.8888539827895412, -6.223338861669884], [0.8789643548422066, -5.45895165611374], [0.8682055578697505, -4.689166334393809], [0.8565882307705718, -3.9147441032119343], [0.8441238614060236, -3.136450754505601], [0.8308247752459663, -2.3550559081896894], [0.8167041194824534, -1.571332251107517], [0.8017758759412279, -0.7860547766472139], [0.8033479854945224, 3.761069677921114e-9]], [[0.9999999999792748], [0.9999999999756509], [0.9999999999708017], [0.9999999999641668], [0.9999999999548312], [0.9999999999412295], [0.999999999920518], [0.9999999998871234], [0.99999999982894

In [ ]:
# recover optimal trajectory

x_sol, u_sol = get_trajectory(solver);